In [1]:
import pandas as pd
import numpy as np
import re
path_to_folder = "C:/Users/phku0/Quant_Project"

### Part 1a company mapping table from 'glassdoorID' to 'cusip' and 'mkvalt'
- many-to-one
- "data_processing/mapping_company.parquet"

In [2]:
df_company = pd.read_parquet(path_to_folder+'/data_raw/company_overview_2024-03-19.parquet')
df_compustat = pd.read_csv(path_to_folder+'/data_raw/compustat_GICS.csv')

In [3]:
# clean df_compustat
df_compustat = df_compustat[['fyear', 'cusip', 'mkvalt', 'conm', 'gsector', 'ggroup', 'gind', 'gsubind']]

df_compustat['cusip'] = df_compustat['cusip'].astype(str).str.lstrip('0').str.slice(0, -1)

df_compustat['gsector'] = df_compustat['gsector'].astype(str).str.split('.').str[0]
df_compustat['ggroup'] = df_compustat['ggroup'].astype(str).str.split('.').str[0]
df_compustat['gind'] = df_compustat['gind'].astype(str).str.split('.').str[0]
df_compustat['gsubind'] = df_compustat['gsubind'].astype(str).str.split('.').str[0]

df_compustat.head()

,fyear,cusip,mkvalt,conm,gsector,ggroup,gind,gsubind
0,1999.0,36110,372.7519,AAR CORP,20,2010,201010,20101010
1,2000.0,36110,377.1180,AAR CORP,20,2010,201010,20101010
2,2001.0,36110,364.5928,AAR CORP,20,2010,201010,20101010
3,2002.0,36110,143.3295,AAR CORP,20,2010,201010,20101010
4,2003.0,36110,308.9071,AAR CORP,20,2010,201010,20101010


In [4]:
# clean df_company
df_company = df_company[['cusip', 'glassdoorId','compustatName']]
df_company['cusip'] = df_company['cusip'].astype(str).str.lstrip('0')
df_company.head()

,cusip,glassdoorId,compustatName
0,30710,656512,AAC HOLDINGS INC
1,30710,4835575,AAC HOLDINGS INC
2,36020,2450,AAON INC
3,36110,4,AAR CORP
4,81T10,13939,ACCO BRANDS CORP


In [5]:
# merge 2 dataframe
df = df_company.merge(df_compustat, on='cusip', how='left')
print(df.shape)
df.head()

(86540, 10)


,cusip,glassdoorId,compustatName,fyear,mkvalt,conm,gsector,ggroup,gind,gsubind
0,30710,656512,AAC HOLDINGS INC,2011.0,NaN,AAC HOLDINGS INC,35,3510,351020,35102020
1,30710,656512,AAC HOLDINGS INC,2012.0,0.0046,AAC HOLDINGS INC,35,3510,351020,35102020
2,30710,656512,AAC HOLDINGS INC,2013.0,4.8000,AAC HOLDINGS INC,35,3510,351020,35102020
3,30710,656512,AAC HOLDINGS INC,2014.0,660.8841,AAC HOLDINGS INC,35,3510,351020,35102020
4,30710,656512,AAC HOLDINGS INC,2015.0,434.8348,AAC HOLDINGS INC,35,3510,351020,35102020


In [6]:
# for rows with the same cusip and glassdoorId, only keep one copy with the highest fyear
df = df.sort_values(by='fyear').drop_duplicates(subset=['cusip', 'glassdoorId'], keep='last')
print(df.shape)
df.head()

(5018, 10)


,cusip,glassdoorId,compustatName,fyear,mkvalt,conm,gsector,ggroup,gind,gsubind
50687,60039Q10,850344,MILLENNIUM SUSTAINABLE V CRP,2006.0,60.6608,MILLENNIUM SUSTAINABLE V CRP,nan,nan,nan,nan
80731,92034410,928,VALPEY-FISHER CORP,2010.0,14.6753,VALPEY-FISHER CORP,20,2010,201040,20104010
38888,45719T10,14816,INHIBITEX INC,2010.0,162.2998,INHIBITEX INC,35,3520,352010,35201010
11854,9534T50,7692,BLUE COAT SYSTEMS INC,2010.0,1269.0432,BLUE COAT SYSTEMS INC,45,4520,452010,45201020
60022,70825420,4993,PENNICHUCK CORP,2010.0,127.9354,PENNICHUCK CORP,55,5510,551040,55104010


In [7]:
# show rows with the same glassdoorId but different cusip
# duplicate_glassdoorId = df.groupby('glassdoorId').filter(lambda x: x['cusip'].nunique() > 1)
# duplicate_glassdoorId = duplicate_glassdoorId.sort_values(by=['glassdoorId'])
# duplicate_glassdoorId.head(10)

In [8]:
df = df.drop_duplicates()
df = df.dropna(subset=['glassdoorId', 'cusip'])
df['mkvalt'] = df['mkvalt'].fillna(-1)

multi_cusip_groups = df.groupby('glassdoorId')['cusip'].nunique()
multi_cusip_groups = multi_cusip_groups[multi_cusip_groups > 1].index
df_clean = df[~((df['glassdoorId'].isin(multi_cusip_groups)) & (df['mkvalt'].isna()))]
df_clean = df_clean.loc[df_clean.groupby('glassdoorId')['mkvalt'].idxmax()]
df_clean = df_clean.reset_index(drop=True)

print(df_clean.shape)
df_clean.head()

(4991, 10)


,cusip,glassdoorId,compustatName,fyear,mkvalt,conm,gsector,ggroup,gind,gsubind
0,36110,4,AAR CORP,2023.0,2533.9171,AAR CORP,20,2010,201010,20101010
1,3110010,7,AMETEK INC,2023.0,38078.0477,AMETEK INC,20,2010,201040,20104010
2,2376R10,8,AMERICAN AIRLINES GROUP INC,2023.0,8989.7110,AMERICAN AIRLINES GROUP INC,20,2030,203020,20302010
3,3215910,9,AMREP CORP,2023.0,106.9170,AMREP CORP,60,6020,602010,60201030
4,282410,12,ABBOTT LABORATORIES,2023.0,190869.7453,ABBOTT LABORATORIES,35,3510,351010,35101010


In [9]:
df2 = df_clean[['cusip', 'glassdoorId', 'mkvalt']]
df2.head()

,cusip,glassdoorId,mkvalt
0,36110,4,2533.9171
1,3110010,7,38078.0477
2,2376R10,8,8989.7110
3,3215910,9,106.9170
4,282410,12,190869.7453


In [10]:
duplicate_ids = df2[df2.duplicated('glassdoorId', keep=False)]
print(duplicate_ids)

Empty DataFrame
Columns: [cusip, glassdoorId, mkvalt]
Index: []


In [11]:
df2 = df2.drop_duplicates()

In [12]:
many_to_one_check = df2.groupby('glassdoorId')['cusip'].nunique()

if (many_to_one_check == 1).all():
  print("OK: There is a many-to-one relationship between 'col1' and 'col2'.")

OK: There is a many-to-one relationship between 'col1' and 'col2'.


In [13]:
df2.to_parquet(path_to_folder+'/data_processing/mapping_company.parquet')

### Part 1b mapping table between 'cusip' and 'GICS' by year
-  one-to-one relationship
- "data_processing/mapping_cusip_GICS.parquet"

In [2]:
df = pd.read_csv(path_to_folder+'/data_raw/compustat_GICS.csv')

In [15]:
# clean df_compustat
df = df[['fyear', 'cusip', 'gsector', 'ggroup', 'gind', 'gsubind']]

df['cusip'] = df['cusip'].astype(str).str.lstrip('0').str.slice(0, -1)

df['gsector'] = df['gsector'].astype(str).str.split('.').str[0]
df['ggroup'] = df['ggroup'].astype(str).str.split('.').str[0]
df['gind'] = df['gind'].astype(str).str.split('.').str[0]
df['gsubind'] = df['gsubind'].astype(str).str.split('.').str[0]

df = df.dropna(subset=['fyear'])
df['fyear'] = df['fyear'].astype(int)
# df['fyear'] = pd.to_datetime(df['fyear'], format='%Y').dt.to_period('Y')

# ensure 1-to-1 relationship
df = df.drop_duplicates(subset=['cusip', 'fyear'], keep='first')

df.head()

,fyear,cusip,gsector,ggroup,gind,gsubind
0,1999,36110,20,2010,201010,20101010
1,2000,36110,20,2010,201010,20101010
2,2001,36110,20,2010,201010,20101010
3,2002,36110,20,2010,201010,20101010
4,2003,36110,20,2010,201010,20101010


In [16]:
df.shape

(275656, 6)

In [17]:
df.to_parquet(path_to_folder+'/data_processing/mapping_cusip_GICS.parquet')

### Part 2 monthly review (score and text)
- scores -> average score by month
- review text -> list of reviews
- export to "data_processing/review_monthly.parquet"

In [2]:
df_glassd_classification = pd.read_parquet(path_to_folder+'/data_raw/glassd_classification_2024-03-19.parquet')
df_glassd_review_internal_data = pd.read_parquet(path_to_folder+'/data_raw/glassd_review_internal_data_2024-03-19.parquet')
df_company_mapping = pd.read_parquet(path_to_folder+'/data_processing/mapping_company.parquet')

In [3]:
# delete irrelevent column
df_glassd_classification.drop(['Id', 'dataVendorId','reviewDetailUrl', 'updateDateTime'], axis=1, inplace=True)
df_glassd_review_internal_data.drop(['updateDateTime', 'other_data'], axis=1, inplace=True)

# join 2 table
df_review = pd.merge(df_glassd_classification, df_glassd_review_internal_data, on='reviewId')

# data cleaning and formatting
df_review['reviewDateTime'] = pd.to_datetime(df_review['reviewDateTime'], errors='coerce')
df_review['month'] = df_review['reviewDateTime'].dt.to_period('M')
df_review.drop(['reviewDateTime'], axis=1, inplace=True)
df_review.drop(['isCurrentJob','lengthOfEmployment', 'jobtitle_ref', 'location_ref', 'countHelpful', 'countNotHelpful'], axis=1, inplace=True)

df_review['ratingBusinessOutlook'] = df_review['ratingBusinessOutlook'].fillna('NEUTRAL').map({
    'POSITIVE': 1,
    'NEUTRAL': 0,
    'NEGATIVE': -1
})
df_review['ratingCeo'] = df_review['ratingCeo'].fillna('NO_OPINION').map({
    'APPROVE': 1,
    'NO_OPINION': 0,
    'DISAPPROVE': -1
})
df_review['ratingRecommendToFriend'] = df_review['ratingRecommendToFriend'].fillna('NO_OPINION').map({
    'POSITIVE': 1,
    'NO_OPINION': 0,
    'NEGATIVE': -1
})

cols = ['summary', 'pros', 'cons', 'advice']
df_review[cols] = df_review[cols].fillna('')

In [4]:
# look up cusip from glassdoorId
df_review = df_review.merge(df_company_mapping, on='glassdoorId', how='left')

print(df_review.shape)
df_review.head(2)

(4337584, 19)


,reviewId,glassdoorId,ratingOverall,ratingCareerOpportunities,ratingCompensationAndBenefits,ratingCultureAndValues,ratingSeniorLeadership,ratingWorkLifeBalance,ratingDiversityAndInclusion,ratingBusinessOutlook,ratingCeo,ratingRecommendToFriend,summary,pros,cons,advice,month,cusip,mkvalt
0,81,1138,5,5,5,0,4,5,0,0,1,1,Apple is unlike any other retail organization.,Apple is very serious about promoting from wit...,"In the end it is still retail. So, there are ...",Keep up the good work.,2008-04,3783310,2.662326e+06
1,83,1138,3,4,4,0,4,2,0,0,1,0,An inside bite into Apple,Apple is a great place to be updated with the ...,As employees we are constantly expected to add...,Sometimes I feel like our company is too forwa...,2008-04,3783310,2.662326e+06


In [5]:
df_review.drop(columns=['summary', 'pros', 'cons', 'advice'], inplace=True)

In [6]:
df_nlp = pd.read_parquet(path_to_folder+'/data_NLP_checkpoints/prod_NLP_cleaned_by_review.parquet')
df = pd.merge(df_review, df_nlp, on='reviewId', how='inner')

In [7]:
print(df.shape)
df.head(2)

(2820648, 36)


,reviewId,glassdoorId,ratingOverall,ratingCareerOpportunities,ratingCompensationAndBenefits,ratingCultureAndValues,ratingSeniorLeadership,ratingWorkLifeBalance,ratingDiversityAndInclusion,ratingBusinessOutlook,...,advice_token_count,summary_sentiment,pros_sentiment,cons_sentiment,advice_sentiment,innovative_llama,integrity_llama,quality_llama,respect_llama,teamwork_llama
0,81,1138,5,5,5,0,4,5,0,0,...,3,0.545637,0.930963,0.000000,0.918141,NaN,NaN,NaN,NaN,0.67
1,83,1138,3,4,4,0,4,2,0,0,...,69,0.000000,0.949141,-0.825076,-0.755065,0.5,0.5,0.67,NaN,0.50


In [8]:
cols = ['ratingOverall', 'ratingCareerOpportunities',
       'ratingCompensationAndBenefits', 'ratingCultureAndValues',
       'ratingSeniorLeadership', 'ratingWorkLifeBalance',
       'ratingDiversityAndInclusion', 'ratingBusinessOutlook']

df[cols] = df[cols].replace(0, np.nan)

In [9]:
df.columns

Index(['reviewId', 'glassdoorId', 'ratingOverall', 'ratingCareerOpportunities',
       'ratingCompensationAndBenefits', 'ratingCultureAndValues',
       'ratingSeniorLeadership', 'ratingWorkLifeBalance',
       'ratingDiversityAndInclusion', 'ratingBusinessOutlook', 'ratingCeo',
       'ratingRecommendToFriend', 'month', 'cusip', 'mkvalt', 'summary_token',
       'pros_token', 'cons_token', 'advice_token', 'summary_length',
       'pros_length', 'cons_length', 'advice_length', 'summary_token_count',
       'pros_token_count', 'cons_token_count', 'advice_token_count',
       'summary_sentiment', 'pros_sentiment', 'cons_sentiment',
       'advice_sentiment', 'innovative_llama', 'integrity_llama',
       'quality_llama', 'respect_llama', 'teamwork_llama'],
      dtype='object')

In [10]:
df.shape

(2820648, 36)

In [11]:
# group by month
# - Calculate mean for numerical columns
# - Concatenate text reviews
aggregation_dict = {
  'ratingOverall': 'mean',
  'ratingCareerOpportunities': 'mean',
  'ratingCompensationAndBenefits': 'mean',
  'ratingCultureAndValues': 'mean',
  'ratingSeniorLeadership': 'mean',
  'ratingWorkLifeBalance': 'mean',
  'ratingDiversityAndInclusion': 'mean',
  'ratingBusinessOutlook': 'mean',
  'ratingCeo': 'mean',
  'ratingRecommendToFriend': 'mean',
  'summary_length': 'mean',
  'pros_length': 'mean',
  'cons_length': 'mean',
  'advice_length': 'mean',
  'summary_token_count': 'mean',
  'pros_token_count': 'mean',
  'cons_token_count': 'mean',
  'advice_token_count': 'mean',
  'summary_sentiment': 'mean',
  'pros_sentiment': 'mean',
  'cons_sentiment': 'mean',
  'advice_sentiment': 'mean',
  'mkvalt': 'mean',
  'summary_token': lambda x: list(x),
  'pros_token': lambda x: list(x),
  'cons_token': lambda x: list(x),
  'advice_token': lambda x: list(x),
  'innovative_llama': 'mean',
  'integrity_llama': 'mean',
  'quality_llama': 'mean',
  'respect_llama': 'mean',
  'teamwork_llama': 'mean'
}

grouped_review_monthly = df.groupby(['month', 'cusip']).agg(aggregation_dict).reset_index()

# round to 2dp
numerical_columns = [
    'ratingOverall', 'ratingCareerOpportunities',
       'ratingCompensationAndBenefits', 'ratingCultureAndValues',
       'ratingSeniorLeadership', 'ratingWorkLifeBalance',
       'ratingDiversityAndInclusion', 'ratingBusinessOutlook', 'ratingCeo',
       'ratingRecommendToFriend', 'month', 'cusip', 'summary_length',
       'pros_length', 'cons_length', 'advice_length', 'summary_token_count',
       'pros_token_count', 'cons_token_count', 'advice_token_count',
       'summary_sentiment', 'pros_sentiment', 'cons_sentiment'
]
grouped_review_monthly[numerical_columns] = grouped_review_monthly[numerical_columns].round(4)

# calculate number of reviews per month per company (shows the number of rows that were grouped together)
grouped_review_count = df.groupby(['month', 'cusip']).size().reset_index(name='review_count')
df_review_monthly = pd.merge(grouped_review_monthly, grouped_review_count, on=['month', 'cusip'])

In [12]:
print(df_review_monthly.shape)
df_review_monthly.head(2)

(292707, 35)


,month,cusip,ratingOverall,ratingCareerOpportunities,ratingCompensationAndBenefits,ratingCultureAndValues,ratingSeniorLeadership,ratingWorkLifeBalance,ratingDiversityAndInclusion,ratingBusinessOutlook,...,summary_token,pros_token,cons_token,advice_token,innovative_llama,integrity_llama,quality_llama,respect_llama,teamwork_llama,review_count
0,2008-04,17275R10,4.6667,4.2222,4.0,NaN,4.3333,4.6667,NaN,NaN,...,"[[amaze, company], [part, cisco, family], [goo...","[[cisco, foster, perfect, environment, technic...","[[nature, work, tends, spawn, people, tech-ori...","[[think, important, understand, supportive, se...",0.678333,0.50,0.76875,0.83,0.737143,9
1,2008-04,17737610,4.5000,4.5000,4.5,NaN,4.5000,4.5000,NaN,NaN,...,"[[best, place, work, south, florida], [citrix,...","[[great, people, great, location, great, work,...","[[none, awesome, honestly, nothing, bad, polit...","[[great, job], [follow, advice, rnrn-when, cho...",0.430000,0.43,0.67000,NaN,0.550000,2


In [13]:
df_review_monthly.to_parquet(path_to_folder+'/data_processing/review_monthly.parquet')

### Part 3 monthly return
- export to "data_processing/price_monthly.parquet"

In [30]:
USstock_daily = pd.read_parquet(path_to_folder+'/data_raw/stock_price_daily.parquet')
print(USstock_daily.shape)
USstock_daily.head()

(25695492, 31)


,cusip,date,shrcd,shrcls,hexcd,trdstat,comnam,ticker,siccd,naics,...,dlprc,cfacpr,cfacshr,vwretd,vwretx,ewretd,ewretx,sprtrn,mcap,mcap_lag1
0,00030710,20141002,11.0,None,1,A,A A C HOLDINGS INC,AAC,6799.0,523910.0,...,NaN,1.0,1.0,0.000776,0.000719,0.003224,0.003202,0.000005,382450500.0,NaN
1,00030710,20141003,11.0,None,1,A,A A C HOLDINGS INC,AAC,6799.0,523910.0,...,NaN,1.0,1.0,0.009103,0.009066,0.004718,0.004702,0.011166,385551450.0,382450500.0
2,00030710,20141006,11.0,None,1,A,A A C HOLDINGS INC,AAC,6799.0,523910.0,...,NaN,1.0,1.0,-0.001585,-0.001612,-0.003940,-0.003952,-0.001565,397748520.0,385551450.0
3,00030710,20141007,11.0,None,1,A,A A C HOLDINGS INC,AAC,6799.0,523910.0,...,NaN,1.0,1.0,-0.015003,-0.015010,-0.012748,-0.012758,-0.015126,397748520.0,397748520.0
4,00030710,20141008,11.0,None,1,A,A A C HOLDINGS INC,AAC,6799.0,523910.0,...,NaN,1.0,1.0,0.015865,0.015631,0.009494,0.009438,0.017462,395474490.0,397748520.0


In [31]:
df_price = USstock_daily[['cusip', 'date', 'vol', 'ret']]

# drop columne with missing return data
df_price = df_price.dropna(subset=['ret'])

# format date
df_price['date'] = pd.to_datetime(df_price['date']).dt.to_period('M')
df_price = df_price.rename(columns={'date': 'month'})

# group by month
df_price_monthly = df_price.groupby(['cusip', 'month']).agg(
  vol_sum=('vol', 'sum'),  # Sum of 'vol'
  monthly_return=('ret', lambda x: (1 + x).prod() - 1)  # Monthly return
).reset_index()

In [32]:
print(df_price_monthly.shape)
df_price_monthly.head()

(1205829, 4)


,cusip,month,vol_sum,monthly_return
0,00030710,2014-10,2632839.0,0.177297
1,00030710,2014-11,2223384.0,0.325069
2,00030710,2014-12,2145445.0,0.071379
3,00030710,2015-01,1243035.0,-0.174644
4,00030710,2015-02,1362187.0,0.410266


In [33]:
df_price_monthly.to_parquet(path_to_folder+'/data_processing/price_monthly.parquet')

### Part 4 Final Merge
- return is 1 month forward than review data
- export to "data_production" folder

In [14]:
df_review = pd.read_parquet(path_to_folder+'/data_processing/review_monthly.parquet')
df_price = pd.read_parquet(path_to_folder+'/data_processing/price_monthly.parquet')
df_GICS = pd.read_parquet(path_to_folder+'/data_processing/mapping_cusip_GICS.parquet')

In [15]:
df_price.shape

(1205829, 4)

In [16]:
# return is 1 month forward than review data
df_price['vol_forward_1m'] = df_price.groupby('cusip')['vol_sum'].shift(-1)
df_price['return_forward_1m'] = df_price.groupby('cusip')['monthly_return'].shift(-1)

df_price = df_price.dropna(subset=['vol_forward_1m', 'return_forward_1m'])
df_price.drop(['vol_sum', 'monthly_return'], axis=1, inplace=True)
# df_price.rename({'vol_sum': 'vol_current', 'monthly_return': 'return_current'})

df_price.head()

,cusip,month,vol_forward_1m,return_forward_1m
0,00030710,2014-10,2223384.0,0.325069
1,00030710,2014-11,2145445.0,0.071379
2,00030710,2014-12,1243035.0,-0.174644
3,00030710,2015-01,1362187.0,0.410266
4,00030710,2015-02,6750656.0,-0.150318


In [17]:
df_merged = pd.merge(df_review, df_price, on=['cusip', 'month'], how='outer')
print(df_merged.shape)
df_merged.head()

(1315030, 37)


,month,cusip,ratingOverall,ratingCareerOpportunities,ratingCompensationAndBenefits,ratingCultureAndValues,ratingSeniorLeadership,ratingWorkLifeBalance,ratingDiversityAndInclusion,ratingBusinessOutlook,...,cons_token,advice_token,innovative_llama,integrity_llama,quality_llama,respect_llama,teamwork_llama,review_count,vol_forward_1m,return_forward_1m
0,2014-10,00030710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2223384.0,0.325069
1,2014-11,00030710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2145445.0,0.071379
2,2014-12,00030710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1243035.0,-0.174644
3,2015-01,00030710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1362187.0,0.410266
4,2015-02,00030710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6750656.0,-0.150318


In [18]:
df_merged.fillna({'review_count': 0}, inplace=True)

df_merged['review_count'] = df_merged['review_count'].astype(int)
df_merged = df_merged.dropna(subset=['return_forward_1m', 'vol_forward_1m'])

print(df_merged.shape)
df_merged.head()

(1193241, 37)


,month,cusip,ratingOverall,ratingCareerOpportunities,ratingCompensationAndBenefits,ratingCultureAndValues,ratingSeniorLeadership,ratingWorkLifeBalance,ratingDiversityAndInclusion,ratingBusinessOutlook,...,cons_token,advice_token,innovative_llama,integrity_llama,quality_llama,respect_llama,teamwork_llama,review_count,vol_forward_1m,return_forward_1m
0,2014-10,00030710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2223384.0,0.325069
1,2014-11,00030710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2145445.0,0.071379
2,2014-12,00030710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1243035.0,-0.174644
3,2015-01,00030710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1362187.0,0.410266
4,2015-02,00030710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,6750656.0,-0.150318


In [19]:
df_GICS.head(2)

,fyear,cusip,gsector,ggroup,gind,gsubind
0,1999,36110,20,2010,201010,20101010
1,2000,36110,20,2010,201010,20101010


In [20]:
# temporary add 'year' column
df_merged['fyear'] = df_merged['month'].dt.year

# lookup GICS
df_merged_GICS = df_merged.merge(df_GICS, on=['cusip', 'fyear'], how='inner')

# remove 'year' column
df_merged_GICS = df_merged_GICS.drop(columns=['fyear'])

In [21]:
df_merged_GICS.head()

,month,cusip,ratingOverall,ratingCareerOpportunities,ratingCompensationAndBenefits,ratingCultureAndValues,ratingSeniorLeadership,ratingWorkLifeBalance,ratingDiversityAndInclusion,ratingBusinessOutlook,...,quality_llama,respect_llama,teamwork_llama,review_count,vol_forward_1m,return_forward_1m,gsector,ggroup,gind,gsubind
0,2000-01,10001P10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,2289998.0,-0.061644,35,3510,351020,35102010
1,2000-02,10001P10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,1098301.0,0.036495,35,3510,351020,35102010
2,2000-03,10001P10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,784418.0,-0.056338,35,3510,351020,35102010
3,2000-04,10001P10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,592542.0,-0.089549,35,3510,351020,35102010
4,2000-05,10001P10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,708249.0,0.213113,35,3510,351020,35102010


In [22]:
df_merged_GICS['gsector'].replace(['NaN', 'nan'], np.nan, inplace=True)
df_merged_GICS = df_merged_GICS.dropna(subset=['gsector'])

C:\Users\phku0\AppData\Local\Temp\ipykernel_41908\1743490492.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged_GICS['gsector'].replace(['NaN', 'nan'], np.nan, inplace=True)


In [24]:
df_merged_GICS.columns

Index(['month', 'cusip', 'ratingOverall', 'ratingCareerOpportunities',
       'ratingCompensationAndBenefits', 'ratingCultureAndValues',
       'ratingSeniorLeadership', 'ratingWorkLifeBalance',
       'ratingDiversityAndInclusion', 'ratingBusinessOutlook', 'ratingCeo',
       'ratingRecommendToFriend', 'summary_length', 'pros_length',
       'cons_length', 'advice_length', 'summary_token_count',
       'pros_token_count', 'cons_token_count', 'advice_token_count',
       'summary_sentiment', 'pros_sentiment', 'cons_sentiment',
       'advice_sentiment', 'mkvalt', 'summary_token', 'pros_token',
       'cons_token', 'advice_token', 'innovative_llama', 'integrity_llama',
       'quality_llama', 'respect_llama', 'teamwork_llama', 'review_count',
       'vol_forward_1m', 'return_forward_1m', 'gsector', 'ggroup', 'gind',
       'gsubind'],
      dtype='object')

In [23]:
df_merged_GICS.to_parquet(path_to_folder+'/data_processing/v3_cleaned.parquet')